📝 **Author:** Amirhossein Heydari - 📧 **Email:** amirhosseinheydari78@gmail.com - 📍 **Linktree:** [linktr.ee/mr_pylin](https://linktr.ee/mr_pylin)

---

# Dependencies

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from sklearn.metrics import classification_report
from torch import nn
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from torch.utils.data import DataLoader, random_split
from torchinfo import summary
from torchmetrics.classification import MulticlassAccuracy, MulticlassConfusionMatrix
from torchvision.datasets import CIFAR10
from torchvision.transforms import v2

In [3]:
# set a seed for deterministic results
seed = 42
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
# check if cuda is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# log
device

# Pre-Processing

# Load Dataset

In [ ]:
# initial transforms
transforms = v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])

# load the CIFAR-10 dataset
trainset = CIFAR10(root="../../datasets", train=True, download=False, transform=transforms)
testset = CIFAR10(root="../../datasets", train=False, download=False, transform=transforms)

# log
print("trainset:")
print(f"    -> trainset.data.shape    : {trainset.data.shape}")
print(f"    -> trainset.data.dtype    : {trainset.data.dtype}")
print(f"    -> type(trainset.data)    : {type(trainset.data)}")
print(f"    -> type(trainset.targets) : {type(trainset.targets)}")
print("-" * 50)
print("testset:")
print(f"    -> testset.data.shape     : {testset.data.shape}")
print(f"    -> testset.data.dtype     : {testset.data.dtype}")
print(f"    -> type(testset.data)     : {type(testset.data)}")
print(f"    -> type(testset.targets)  : {type(testset.targets)}")
print("-" * 50)
print(f"classes: {trainset.class_to_idx}")
print(f"trainset distribution: {np.unique(trainset.targets, return_counts=True)[1]}")
print(f"testset  distribution: {np.unique(testset.targets, return_counts=True)[1]}")

In [ ]:
# plot
fig, axs = plt.subplots(nrows=4, ncols=8, figsize=(12, 6), layout="compressed")
for i in range(4):
    for j in range(8):
        axs[i, j].imshow(trainset.data[i * 8 + j], cmap="gray")
        axs[i, j].set_title(trainset.classes[trainset.targets[i * 8 + j]])
        axs[i, j].axis("off")
plt.show()

## Split trainset into [trainset, validationset]

In [ ]:
# random split
trainset, validationset = random_split(trainset, [0.9, 0.1])

# log
print("trainset:")
print(f"    -> len(trainset)       : {len(trainset)}")
print(f"    -> trainset[0][0]      : {trainset[0][0].shape}")
print(f"    -> trainset[0][1]      : {trainset[0][1]}\n")
print("validationset:")
print(f"    -> len(validationset)  : {len(validationset)}")
print(f"    -> validationset[0][0] : {validationset[0][0].shape}")
print(f"    -> validationset[0][1] : {validationset[0][1]}\n")
print("testset:")
print(f"    -> len(testset)        : {len(testset)}")
print(f"    -> testset[0][0]       : {testset[0][0].shape}")
print(f"    -> testset[0][1]       : {testset[0][1]}")

## Normalization

In [ ]:
# create a temporary DataLoader for the trainset
temp_trainloader = DataLoader(trainset, batch_size=len(trainset))

# get the whole data
temp_dataset = next(iter(temp_trainloader))

# calculate the mean and standard deviation [PER CHANNEL]
train_mean = temp_dataset[0].mean(axis=(0, 2, 3))  # [0.4917, 0.4823, 0.4467]
train_std = temp_dataset[0].std(axis=(0, 2, 3))  # [0.2471, 0.2435, 0.2616]

del temp_trainloader
del temp_dataset

# log
print(f"train mean per channel : {train_mean}")
print(f"train std  per channel : {train_std}")

## Transform
   - on-the-fly data augmentation
   - Disadvantage:
      - same transform applies to the same data in each epoch
   - Advantage:
      - Reduced Memory Usage, Regularization & Data Diversity [random transforms e.g. RancomCrop]

In [ ]:
transforms

In [ ]:
transforms.transforms.append(v2.Normalize(mean=train_mean, std=train_std))

# log
print(f"trainset.dataset.transforms:\n{trainset.dataset.transforms}\n")
print(f"validationset.dataset.transforms:\n{validationset.dataset.transforms}\n")
print(f"testset.transforms:\n{testset.transforms}")

In [ ]:
# log
print("before applying transform:")
print(f"    -> type(testset.data[0]) : {type(testset.data[0])}")
print(f"    -> testset.data[0].dtype : {testset.data[0].dtype}")
print(f"    -> testset.data[0].shape : {testset.data[0].shape}")
print("-" * 50)
print("after applying transform:")
print(f"    -> type(testset[0][0])   : {type(testset[0][0])}")
print(f"    -> testset[0][0].dtype   : {testset[0][0].dtype}")
print(f"    -> testset[0][0].shape   : {testset[0][0].shape}")

## DataLoader

In [12]:
batch_size = 64

trainloader = DataLoader(dataset=trainset, batch_size=batch_size, shuffle=True, num_workers=2)
validationloader = DataLoader(dataset=validationset, batch_size=batch_size, shuffle=False, num_workers=2)
testloader = DataLoader(dataset=testset, batch_size=batch_size, shuffle=False, num_workers=2)

In [ ]:
first_train_batch = next(iter(trainloader))
first_validation_batch = next(iter(validationloader))
first_test_batch = next(iter(testloader))

print(
    f"trainloader      first batch     -> x.shape: {first_train_batch[0].shape} - y.shape: {first_train_batch[1].shape} - x.dtype: {first_train_batch[0].dtype} - y.dtype: {first_train_batch[1].dtype}"
)
print(
    f"validationloader first batch     -> x.shape: {first_validation_batch[0].shape} - y.shape: {first_validation_batch[1].shape} - x.dtype: {first_validation_batch[0].dtype} - y.dtype: {first_validation_batch[1].dtype}"
)
print(
    f"testloader       first batch     -> x.shape: {first_test_batch[0].shape} - y.shape: {first_test_batch[1].shape} - x.dtype: {first_test_batch[0].dtype} - y.dtype: {first_test_batch[1].dtype}"
)
print(f"trainloader      last batch-size -> {len(trainset) % batch_size}")
print(f"validationloader last batch-size -> {len(validationset) % batch_size}")
print(f"testloader       last batch-size -> {len(testset) % batch_size}")

# Network Structure: Convolutional Neural Networks

<figure style="text-align: center;">
    <img src="../../assets/images/original/cnn/convolutional-neural-networks.svg" alt="convolutional-neural-networks.svg" style="width: 100%;">
    <figcaption>Convolutional Neural Networks Model</figcaption>
</figure>

In [14]:
class CIFAR10Model(nn.Module):
    def __init__(self, in_channels, output_dim):
        super(CIFAR10Model, self).__init__()
        self.feature_extractor = nn.Sequential(
            # 3x32x32
            nn.Conv2d(in_channels, out_channels=32, kernel_size=3),
            nn.BatchNorm2d(32),  # StandardScaler along channel axis
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            # 32x15x15
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            # 64x6x6
            nn.AdaptiveAvgPool2d(output_size=(1, 1)),
            # 64x1x1
        )

        self.flatten = nn.Flatten(start_dim=1)

        self.classifier = nn.Sequential(
            nn.Linear(64, output_dim),
        )

    def forward(self, x):
        x = self.feature_extractor(x)
        x = self.flatten(x)
        x = self.classifier(x)
        return x

In [ ]:
in_channels = trainset[0][0].shape[0]
output_dim = len(trainset.dataset.classes)

# initialize the model
model = CIFAR10Model(in_channels, output_dim).to(device)

# log
model

In [ ]:
summary(model, input_size=(batch_size, *testset.data.transpose(0, 3, 1, 2).shape[1:]))

# Set up remaining Hyperparameters

In [17]:
lr = 0.01
criterion = CrossEntropyLoss()
optimizer = Adam(params=model.parameters(), lr=lr)
num_epochs = 10

# Train & Validation Loop

In [18]:
train_acc_per_epoch = []
train_loss_per_epoch = []
val_acc_per_epoch = []
val_loss_per_epoch = []

In [19]:
train_acc = MulticlassAccuracy(num_classes=len(testset.classes), top_k=1).to(device)
val_acc = MulticlassAccuracy(num_classes=len(testset.classes), top_k=1).to(device)

In [ ]:
for epoch in range(num_epochs):

    # train loop
    model.train()
    train_loss = 0

    for x, y in trainloader:

        # send data to GPU
        x, y_true = x.to(device), y.to(device)

        # forward
        y_pred = model(x)
        loss = criterion(y_pred, y_true)

        # backward
        loss.backward()

        # update parameters
        optimizer.step()
        optimizer.zero_grad()

        # store loss and accuracy per iteration
        train_loss += loss.item() * len(x)
        train_acc.update(y_pred, y_true)

    # store loss and accuracy per epoch
    train_loss_per_epoch.append(train_loss / len(trainset))
    train_acc_per_epoch.append(train_acc.compute().item())
    train_acc.reset()

    # validation loop
    model.eval()
    val_loss = 0

    with torch.no_grad():
        for x, y in validationloader:

            # send data to GPU
            x, y_true = x.to(device), y.to(device)

            # forward
            y_pred = model(x)
            loss = criterion(y_pred, y_true)

            # store loss and accuracy per iteration
            val_loss += loss.item() * len(x)
            val_acc.update(y_pred, y_true)

    # store loss and accuracy per epoch
    val_loss_per_epoch.append(val_loss / len(validationset))
    val_acc_per_epoch.append(val_acc.compute().item())
    val_acc.reset()

    # log
    print(
        f"epoch {epoch+1:0{len(str(num_epochs))}}/{num_epochs}  ->  train[loss: {train_loss_per_epoch[epoch]:.5f} - acc: {train_acc_per_epoch[epoch]:.2f}] | validation[loss: {val_loss_per_epoch[epoch]:.5f} - acc: {val_acc_per_epoch[epoch]:.2f}]"
    )

## Model Analysis

In [ ]:
# plot
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 4), layout="compressed")
axs[0].plot(train_loss_per_epoch, label="Train loss")
axs[0].plot(val_loss_per_epoch, label="Validation loss")
axs[0].set(title="Loss over time", xlabel="Epoch", ylabel="Loss")
axs[0].legend(loc="best", fancybox=True, shadow=True)
axs[1].plot(train_acc_per_epoch, label="Train accuracy")
axs[1].plot(val_acc_per_epoch, label="Validation accuracy")
axs[1].set(title="Accuracy over time", xlabel="Epoch", ylabel="Accuracy")
axs[1].legend(loc="best", fancybox=True, shadow=True)
plt.show()

# Test Loop

In [22]:
test_acc = MulticlassAccuracy(num_classes=len(testset.classes), top_k=1).to(device)

In [ ]:
model.eval()
test_loss = 0
predictions = []
targets = []

with torch.no_grad():
    for x, y in testloader:

        # send data to GPU
        x, y_true = x.to(device), y.to(device)

        # forward
        y_pred = model(x)
        loss = criterion(y_pred, y_true)

        # store loss and accuracy per iteration
        test_loss += loss.item() * len(x)
        test_acc.update(y_pred, y_true)

        predictions.extend(y_pred.argmax(dim=1).cpu())
        targets.extend(y_true.cpu())

# log
print(f"test[loss: {test_loss / len(testset):.5f} - acc: {test_acc.compute().item():.2f}]")

## Metrics
   - Loss
   - Accuracy
   - Recall
   - Precision
   - F1-Score
   - Confusion Matrix
   - Area Under the ROC Curve (AUC-ROC)
   - Area Under the Precision-Recall Curve (AUC-PR)
   - ...

**Docs**:
   - [lightning.ai/docs/torchmetrics/stable/all-metrics.html](https://lightning.ai/docs/torchmetrics/stable/all-metrics.html)
   - [scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)

In [ ]:
# classification report
print(classification_report(targets, predictions))

In [ ]:
# confusion matrix
metric = MulticlassConfusionMatrix(num_classes=10)
confusion_matrix = metric(torch.tensor(predictions), torch.tensor(targets))

# log
print(confusion_matrix)

# plot
fig, ax = plt.subplots(figsize=(8, 8))
metric.plot(ax=ax)
plt.show()

# Prediction

In [28]:
def predict(model: nn.Module, data: np.ndarray, classes: list, transform: v2._container.Compose = None) -> torch.Tensor:

    # add batch dimension to a single data
    if len(data.shape) == 3:
        data = np.expand_dims(data, axis=0)

    # apply the transform
    if transform:
        data = torch.stack([transform(sample) for sample in data])

    # predict
    model.eval()
    with torch.no_grad():

        # send data to GPU
        data = data.to(device)

        # forward
        y_pred = model(data).argmax(dim=1).cpu()

        # idx to labels
        y_pred = np.array(classes)[y_pred]

    return y_pred

In [ ]:
# some raw data
raw_data = CIFAR10(root="../../datasets", train=False, transform=None, download=False).data[:32]

# predict
y_pred = predict(model, data=raw_data, classes=testset.classes, transform=transforms)

# log
print(f"predictions:\n{y_pred}")

In [ ]:
# plot
fig, axs = plt.subplots(nrows=4, ncols=8, figsize=(12, 6), layout="compressed")
for i in range(4):
    for j in range(8):
        axs[i, j].imshow(raw_data[i * 8 + j], cmap="gray")
        axs[i, j].set_title(predict(model, raw_data[i * 8 + j], testset.classes, transform=transforms))
        axs[i, j].axis("off")
plt.show()